In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from glob import glob
from tqdm import tqdm_notebook as tqdm

In [18]:
glob(label_fp.replace('label','Faked_*'))
label_fp.replace('label','Faked_*')

'/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199176-labFaked_*1.jpg'

In [25]:
full_fps = glob('/home/rob/HWR/datasets/Aarhus/*lab.jpg')
for full_fp in tqdm(full_fps):
    full_faked_fp = full_fp.replace('lab.jpg','FullFaked.jpg')
    if os.path.exists(full_faked_fp):
        continue
        
    cropped = cv2.imread(full_fp)
    label_fps = glob(full_fp.replace('.jpg','label*.jpg'))
    for label_fp in label_fps:
        label_row = df[df['orig_fp']==label_fp].iloc[0]
        faked_fps = glob(label_fp.replace('label','Faked_*'))
        if not len(faked_fps):
            #print(label_fp.replace('label','Faked_*'))
            continue
        
        faked_fp = np.random.choice(faked_fps)
        _type = faked_fp.split('_')[-1][:-5]
        if _type == 'QR':
            continue
        faked = cv2.imread(faked_fp)
        angle = label_row['angle']
        index = label_row.name
        sx,sy,ex,ey = label_row['sy'],label_row['sx'],label_row['ey'],label_row['ex']
        avg_color = faked.mean(axis=(0,1)).astype('uint8').astype('float')
        faked_rotated = rotate_bound(faked,-angle,avg_color)
        ex = min(min(ex-sx,faked_rotated.shape[0]) + sx,cropped.shape[0])
        ey = min(min(ey-sy,faked_rotated.shape[1]) + sy,cropped.shape[1])
        for i in range(sx,ex):
            x = i-sx
            for j in range(sy,ey):
                y = j-sy
                if not (faked_rotated[x,y] == avg_color).all():
                    cropped[i,j] = faked_rotated[x,y] 
        
        df.loc[index,'type'] = _type
        df.to_csv('box coords aarhus.csv')
        
    cv2.imwrite(full_faked_fp, cropped)

In [30]:

df.to_csv('box coords aarhus.csv')

In [91]:
df.to_csv('box coords.csv')

In [83]:
ex-sx,ey-sy,faked_rotated.shape

(1440, 1624, (1440, 1624, 3))

In [86]:
i,j,cropped.shape

(2902, 53, (2902, 3327, 3))

In [4]:
def rotate_bound(image, angle, borderValue):
    # grab the dimensions of the image and then determine the
    # center
    (h, w) = image.shape[:2]
    (cX, cY) = (w / 2, h / 2)

    # grab the rotation matrix (applying the negative of the
    # angle to rotate clockwise), then grab the sine and cosine
    # (i.e., the rotation components of the matrix)
    M = cv2.getRotationMatrix2D((cX, cY), angle, 1.0)
    cos = np.abs(M[0, 0])
    sin = np.abs(M[0, 1])

    # compute the new bounding dimensions of the image
    nW = int((h * sin) + (w * cos))
    nH = int((h * cos) + (w * sin))

    # adjust the rotation matrix to take into account translation
    M[0, 2] += (nW / 2) - cX
    M[1, 2] += (nH / 2) - cY

    # perform the actual rotation and return the image
    return cv2.warpAffine(image, M, (nW, nH), borderValue=borderValue)

In [3]:
df = pd.read_csv('box coords aarhus.csv', index_col=0)
df['type'] = df['type'].map(lambda x: 'QR' if x == 'QR Code' else x)
df

,catalog_no,id,sx,sy,ex,ey,orig_fp,angle,type
0,NHMA-ENT-199269-lab.jpg,0,217,214,526,410,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199269-...,2.868719,NaN
1,NHMA-ENT-199269-lab.jpg,1,553,259,867,440,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199269-...,4.749293,NaN
2,NHMA-ENT-199269-lab.jpg,2,362,450,654,630,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199269-...,5.759731,NaN
3,NHMA-ENT-199244-lab.jpg,0,296,286,812,489,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199244-...,-1.551053,NaN
4,NHMA-ENT-199244-lab.jpg,1,440,531,621,655,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199244-...,-1.051181,NaN
5,NHMA-ENT-199370-lab.jpg,0,257,60,771,251,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199370-...,2.846707,NaN
6,NHMA-ENT-199370-lab.jpg,1,116,225,833,547,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199370-...,2.871674,NaN
7,NHMA-ENT-199370-lab.jpg,2,396,553,633,679,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199370-...,-1.037840,NaN
8,NHMA-ENT-199028-lab.jpg,0,194,178,521,413,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199028-...,3.660059,NaN
9,NHMA-ENT-199028-lab.jpg,1,525,239,827,408,/home/rob/HWR/datasets/Aarhus/NHMA-ENT-199028-...,2.569686,NaN
